In [2]:
import os
import surprise
from surprise import Reader, Dataset

file_path = os.path.expanduser('/Users/drakaris/Desktop/personalisation projects/personalization-theory-master/sampled_data.csv')
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_file(file_path, reader=reader)
data.split(n_folds=5) # data can now be used normally

In [3]:
from surprise import SVD, evaluate, GridSearch
param_grid = {'n_epochs': [20, 25, 30], 'lr_all': [0.01, 0.012],
              'reg_all': [0.04, 0.06]}
grid_search = GridSearch(SVD, param_grid, measures=['RMSE', 'MAE'])
grid_search.evaluate(data)

import pandas as pd  # noqa

results_svd = pd.DataFrame.from_dict(grid_search.cv_results)
print(results_svd)

# best RMSE score
print(grid_search.best_score['RMSE'])
# >>> 1.17046722149

# combination of parameters that gave the best RMSE score
print(grid_search.best_params['RMSE'])
# >>> {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 25}

# best MAE score
print(grid_search.best_score['MAE'])
# >>> 0.909007988848

# combination of parameters that gave the best MAE score
print(grid_search.best_params['MAE'])
# >>> {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 30}



[{'lr_all': 0.01, 'reg_all': 0.04, 'n_epochs': 20}, {'lr_all': 0.01, 'reg_all': 0.04, 'n_epochs': 25}, {'lr_all': 0.01, 'reg_all': 0.04, 'n_epochs': 30}, {'lr_all': 0.01, 'reg_all': 0.06, 'n_epochs': 20}, {'lr_all': 0.01, 'reg_all': 0.06, 'n_epochs': 25}, {'lr_all': 0.01, 'reg_all': 0.06, 'n_epochs': 30}, {'lr_all': 0.012, 'reg_all': 0.04, 'n_epochs': 20}, {'lr_all': 0.012, 'reg_all': 0.04, 'n_epochs': 25}, {'lr_all': 0.012, 'reg_all': 0.04, 'n_epochs': 30}, {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 20}, {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 25}, {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 30}]
------------
Parameters combination 1 of 12
params:  {'lr_all': 0.01, 'reg_all': 0.04, 'n_epochs': 20}
------------
Mean RMSE: 1.1819
Mean MAE : 0.9183
------------
------------
Parameters combination 2 of 12
params:  {'lr_all': 0.01, 'reg_all': 0.04, 'n_epochs': 25}
------------
Mean RMSE: 1.1820
Mean MAE : 0.9169
------------
------------
Parameters combination 3 of 12
params:

In [4]:
from surprise import NMF, evaluate
param_grid = {'n_factors': [20, 25, 30],'n_epochs': [30, 40, 50] }
grid_search = GridSearch(NMF, param_grid, measures=['RMSE', 'MAE'])
grid_search.evaluate(data)

import pandas as pd  # noqa

results_nmf = pd.DataFrame.from_dict(grid_search.cv_results)
print(results_nmf)

# best RMSE score
print(grid_search.best_score['RMSE'])
# >>> 1.17046722149

# combination of parameters that gave the best RMSE score
print(grid_search.best_params['RMSE'])
# >>> {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 25}

# best MAE score
print(grid_search.best_score['MAE'])
# >>> 0.909007988848

# combination of parameters that gave the best MAE score
print(grid_search.best_params['MAE'])
# >>> {'lr_all': 0.012, 'reg_all': 0.06, 'n_epochs': 30}



[{'n_factors': 20, 'n_epochs': 30}, {'n_factors': 20, 'n_epochs': 40}, {'n_factors': 20, 'n_epochs': 50}, {'n_factors': 25, 'n_epochs': 30}, {'n_factors': 25, 'n_epochs': 40}, {'n_factors': 25, 'n_epochs': 50}, {'n_factors': 30, 'n_epochs': 30}, {'n_factors': 30, 'n_epochs': 40}, {'n_factors': 30, 'n_epochs': 50}]
------------
Parameters combination 1 of 9
params:  {'n_factors': 20, 'n_epochs': 30}
------------
Mean RMSE: 1.2818
Mean MAE : 0.9923
------------
------------
Parameters combination 2 of 9
params:  {'n_factors': 20, 'n_epochs': 40}
------------
Mean RMSE: 1.2839
Mean MAE : 1.0048
------------
------------
Parameters combination 3 of 9
params:  {'n_factors': 20, 'n_epochs': 50}
------------
Mean RMSE: 1.2898
Mean MAE : 1.0100
------------
------------
Parameters combination 4 of 9
params:  {'n_factors': 25, 'n_epochs': 30}
------------
Mean RMSE: 1.2756
Mean MAE : 0.9725
------------
------------
Parameters combination 5 of 9
params:  {'n_factors': 25, 'n_epochs': 40}
------

In [108]:
algo= SVD()
predictions=[]
for trainset, testset in data.folds():
    algo.train(trainset)
    predictions=predictions+testset 

i=0    

print(len(predictions))
useridlist=[]
itemidlist=[]
finallistofest=[]
topfive=[[]  for i in range(2915)]

x = [[] for i in range(3)]
k=0
for counter in range(0,len(predictions)):
    userid=predictions[counter][0]
    itemidlist=[]
    #print(userid)
    if userid not in useridlist:
        useridlist.append(userid)
        for thing in range (0,len(predictions)):
            itemid=predictions[thing][1]
            if itemid not in itemidlist:
                i=i+1
                #print(i)
                #print(userid)
                #print(itemid)
                itemidlist.append(itemid)
                finallistofest.append([algo.predict(userid, itemid).iid,algo.predict(userid, itemid).est])
        finallistofest.sort(key=lambda x: x[1], reverse=True)
        topfive[k]=finallistofest[:5]        
        k=k+1        

print(len(useridlist))
print(len(topfive))
            




3383
2915
2915
